# Penalized-Constrained Regression (PCReg) Simulation Study Findings

This notebook contains reproducible analysis of the Monte Carlo simulation comparing PCReg against OLS and other regression methods for learning curve estimation.

**Key Finding**: PCReg with constraints-only (alpha=0) outperforms OLS in **58.2%** of scenarios overall, with performance advantage strongest when:
- CV error is low (data quality is high)
- Sample size is small to medium (n = 5-10 lots)
- OLS produces coefficients with wrong signs (PCReg wins 81% in these cases)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Load simulation results
RESULTS_PATH = Path('output_v2/simulation_results.parquet')
df = pd.read_parquet(RESULTS_PATH)

print(f"Total observations: {len(df):,}")
print(f"Models: {df['model_name'].nunique()}")
print(f"Scenarios: {len(df) // df['model_name'].nunique():,}")

## 1. Data Generation Verification

Verify that the correlation between log(midpoint) and log(quantity) matches target values.

In [ ]:
# Check actual vs target correlation
corr_check = df[df['model_name'] == 'OLS'].groupby('target_correlation')['actual_correlation'].agg(['mean', 'std', 'min', 'max'])
print("Target vs Actual Correlation:")
display(corr_check.round(3))

## 2. Why OLS and PCReg Are Different Models

OLS and PCReg are **fundamentally different** models:

| Aspect | OLS | PCReg |
|--------|-----|-------|
| **Working space** | Log-log space | Unit space |
| **Loss function** | MSE on log(Y) | SSPE on Y |
| **Model form** | log(Y) = a + b*log(X1) + c*log(X2) | Y = T1 * X1^b * X2^c |

Even with identical coefficients (b, c), these models optimize different objectives.

In [ ]:
# Compare OLS vs PCReg_ConstrainOnly (alpha=0)
ols = df[df['model_name'] == 'OLS'].copy()
pcreg = df[df['model_name'] == 'PCReg_ConstrainOnly'].copy()

# Create merge key
def make_key(r):
    return f"{r['n_lots']}_{r['target_correlation']}_{r['cv_error']}_{r['learning_rate']}_{r['rate_effect']}_{r['replication']}"

ols['key'] = ols.apply(make_key, axis=1)
pcreg['key'] = pcreg.apply(make_key, axis=1)

merged = ols.set_index('key')[['test_sspe', 'b', 'c', 'b_correct_sign', 'c_correct_sign', 'n_lots', 'target_correlation', 'cv_error']].rename(
    columns={'test_sspe': 'ols_sspe', 'b': 'ols_b', 'c': 'ols_c'}
)
merged = merged.join(pcreg.set_index('key')[['test_sspe', 'b', 'c']].rename(
    columns={'test_sspe': 'pcreg_sspe', 'b': 'pcreg_b', 'c': 'pcreg_c'}
))
merged['pcreg_wins'] = merged['pcreg_sspe'] < merged['ols_sspe']
merged['any_wrong_sign'] = ~(merged['b_correct_sign'] & merged['c_correct_sign'])

print("Overall: PCReg_ConstrainOnly vs OLS")
print(f"  PCReg wins: {merged.pcreg_wins.sum():,} ({100*merged.pcreg_wins.mean():.1f}%)")
print(f"  OLS wins: {(~merged.pcreg_wins).sum():,} ({100*(~merged.pcreg_wins).mean():.1f}%)")

## 3. Sign Correctness Analysis

Learning curve slopes should be negative (b ≤ 0, c ≤ 0). When does OLS produce wrong signs?

In [ ]:
# Sign analysis by model
print("Sign Correctness by Model:")
print("="*60)
for model in ['OLS', 'Ridge', 'Lasso', 'BayesianRidgeModel', 'PCReg_ConstrainOnly', 'PCReg_CV']:
    m = df[df['model_name'] == model]
    both_correct = (m.b_correct_sign & m.c_correct_sign).mean()
    print(f"  {model:20s}: {100*both_correct:.1f}% both signs correct")

print("\nNote: PCReg always has correct signs because of constraints!")

In [ ]:
# Wrong sign frequency by condition
print("OLS Wrong Sign Rate by Condition:")
print("="*60)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# By n_lots
wrong_by_n = merged.groupby('n_lots')['any_wrong_sign'].mean()
axes[0].bar(wrong_by_n.index.astype(str), wrong_by_n.values)
axes[0].set_xlabel('n_lots')
axes[0].set_ylabel('Wrong Sign Rate')
axes[0].set_title('Wrong Sign Rate by Sample Size')
for i, v in enumerate(wrong_by_n.values):
    axes[0].text(i, v + 0.01, f'{v:.1%}', ha='center')

# By correlation
wrong_by_corr = merged.groupby('target_correlation')['any_wrong_sign'].mean()
axes[1].bar(wrong_by_corr.index.astype(str), wrong_by_corr.values)
axes[1].set_xlabel('Correlation')
axes[1].set_ylabel('Wrong Sign Rate')
axes[1].set_title('Wrong Sign Rate by Correlation')
for i, v in enumerate(wrong_by_corr.values):
    axes[1].text(i, v + 0.01, f'{v:.1%}', ha='center')

# By cv_error
wrong_by_cv = merged.groupby('cv_error')['any_wrong_sign'].mean()
axes[2].bar(wrong_by_cv.index.astype(str), wrong_by_cv.values)
axes[2].set_xlabel('CV Error')
axes[2].set_ylabel('Wrong Sign Rate')
axes[2].set_title('Wrong Sign Rate by CV Error')
for i, v in enumerate(wrong_by_cv.values):
    axes[2].text(i, v + 0.01, f'{v:.1%}', ha='center')

plt.tight_layout()
plt.show()

In [ ]:
# Wrong sign heatmap: n_lots x correlation
pivot = merged.pivot_table(values='any_wrong_sign', index='n_lots', columns='target_correlation', aggfunc='mean')

plt.figure(figsize=(8, 5))
sns.heatmap(pivot, annot=True, fmt='.1%', cmap='Reds', cbar_kws={'label': 'Wrong Sign Rate'})
plt.title('OLS Wrong Sign Rate by n_lots × Correlation')
plt.xlabel('Target Correlation')
plt.ylabel('n_lots (Sample Size)')
plt.show()

print("Key Finding: Wrong signs occur most often when:")
print("  - Small sample size (n=5): 14.4%")
print("  - High correlation (0.9): 11.3%")
print("  - High CV error (0.20): 14.2%")

## 4. PCReg Performance When OLS Has Wrong Signs

This is where PCReg really shines - when OLS produces unreasonable coefficients.

In [ ]:
# Compare performance conditional on sign correctness
wrong_sign = merged[merged['any_wrong_sign']]
correct_sign = merged[~merged['any_wrong_sign']]

print("="*70)
print("MODEL COMPARISON CONDITIONAL ON OLS SIGN CORRECTNESS")
print("="*70)
print()
print(f"Total scenarios with OLS wrong sign: {len(wrong_sign):,} ({100*len(wrong_sign)/len(merged):.1f}%)")
print()
print("When OLS has WRONG sign:")
print(f"  PCReg wins: {wrong_sign.pcreg_wins.sum()} / {len(wrong_sign)} ({100*wrong_sign.pcreg_wins.mean():.1f}%)")
print(f"  OLS wins: {(~wrong_sign.pcreg_wins).sum()} / {len(wrong_sign)} ({100*(~wrong_sign.pcreg_wins).mean():.1f}%)")
print()
print("When OLS has CORRECT sign:")
print(f"  PCReg wins: {correct_sign.pcreg_wins.sum()} / {len(correct_sign)} ({100*correct_sign.pcreg_wins.mean():.1f}%)")
print(f"  OLS wins: {(~correct_sign.pcreg_wins).sum()} / {len(correct_sign)} ({100*(~correct_sign.pcreg_wins).mean():.1f}%)")
print()
print("KEY INSIGHT: PCReg wins 81% when OLS has wrong signs vs 57% with correct signs!")

In [ ]:
# Visualize the difference
fig, ax = plt.subplots(figsize=(8, 5))

categories = ['OLS Wrong Sign\n(n=399)', 'OLS Correct Sign\n(n=5,676)']
pcreg_wins = [wrong_sign.pcreg_wins.mean() * 100, correct_sign.pcreg_wins.mean() * 100]
ols_wins = [100 - pcreg_wins[0], 100 - pcreg_wins[1]]

x = np.arange(len(categories))
width = 0.35

bars1 = ax.bar(x - width/2, pcreg_wins, width, label='PCReg Wins', color='steelblue')
bars2 = ax.bar(x + width/2, ols_wins, width, label='OLS Wins', color='coral')

ax.set_ylabel('Win Rate (%)')
ax.set_title('PCReg vs OLS Win Rate by OLS Sign Correctness')
ax.set_xticks(x)
ax.set_xticklabels(categories)
ax.legend()
ax.axhline(y=50, color='gray', linestyle='--', alpha=0.5)

# Add value labels
for bar in bars1:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, f'{bar.get_height():.1f}%', ha='center')
for bar in bars2:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, f'{bar.get_height():.1f}%', ha='center')

plt.tight_layout()
plt.show()

## 5. Does Penalization (alpha > 0) Help?

Compare PCReg with CV-tuned alpha vs constraints-only (alpha=0).

In [ ]:
# Alpha selection distribution
pcreg_cv = df[df['model_name'] == 'PCReg_CV']

print("PCReg_CV Alpha Selection Distribution:")
print("="*50)
alpha_dist = pcreg_cv['alpha'].value_counts().sort_index()
for alpha, count in alpha_dist.items():
    print(f"  alpha={alpha:.6f}: {count:,} ({100*count/len(pcreg_cv):.1f}%)")

In [ ]:
# Compare PCReg_CV vs PCReg_ConstrainOnly
pcreg_cv = df[df['model_name'] == 'PCReg_CV'].copy()
pcreg_constrain = df[df['model_name'] == 'PCReg_ConstrainOnly'].copy()

pcreg_cv['key'] = pcreg_cv.apply(make_key, axis=1)
pcreg_constrain['key'] = pcreg_constrain.apply(make_key, axis=1)

cv_comparison = pcreg_constrain.set_index('key')[['test_sspe']].rename(columns={'test_sspe': 'constrain_sspe'})
cv_comparison = cv_comparison.join(pcreg_cv.set_index('key')[['test_sspe', 'alpha']].rename(columns={'test_sspe': 'cv_sspe'}))
cv_comparison['cv_wins'] = cv_comparison['cv_sspe'] < cv_comparison['constrain_sspe']

print("PCReg_CV (with alpha>0 option) vs PCReg_ConstrainOnly (alpha=0):")
print(f"  PCReg_CV wins: {cv_comparison.cv_wins.sum():,} ({100*cv_comparison.cv_wins.mean():.1f}%)")
print(f"  ConstrainOnly wins: {(~cv_comparison.cv_wins).sum():,} ({100*(~cv_comparison.cv_wins).mean():.1f}%)")
print()
print("Conclusion: Constraints alone (alpha=0) often outperform CV-tuned penalties!")

## 6. Decision Rules Using Observable Factors

In practice, users can observe:
- **n_lots**: Sample size
- **correlation**: Correlation between predictors (estimable from data)
- **cv_error**: Data noise (estimable from residuals)

They **cannot** observe learning_rate or rate_effect (true but unknown parameters).

In [ ]:
# Win rate tables by observable factors
print("PCReg Win Rate by n_lots × cv_error:")
pivot1 = merged.pivot_table(values='pcreg_wins', index='n_lots', columns='cv_error', aggfunc='mean')
display((pivot1 * 100).round(1).astype(str) + '%')

print("\nPCReg Win Rate by n_lots × correlation:")
pivot2 = merged.pivot_table(values='pcreg_wins', index='n_lots', columns='target_correlation', aggfunc='mean')
display((pivot2 * 100).round(1).astype(str) + '%')

In [ ]:
# Heatmap of win rates
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# n_lots x cv_error
sns.heatmap(pivot1 * 100, annot=True, fmt='.1f', cmap='RdYlGn', center=50, ax=axes[0],
            cbar_kws={'label': 'PCReg Win Rate (%)'})
axes[0].set_title('PCReg Win Rate: n_lots × cv_error')
axes[0].set_xlabel('CV Error')
axes[0].set_ylabel('n_lots')

# n_lots x correlation
sns.heatmap(pivot2 * 100, annot=True, fmt='.1f', cmap='RdYlGn', center=50, ax=axes[1],
            cbar_kws={'label': 'PCReg Win Rate (%)'})
axes[1].set_title('PCReg Win Rate: n_lots × correlation')
axes[1].set_xlabel('Target Correlation')
axes[1].set_ylabel('n_lots')

plt.tight_layout()
plt.show()

In [ ]:
# Decision tree with observable factors only
from sklearn.tree import DecisionTreeClassifier, export_text

X_observable = merged[['n_lots', 'target_correlation', 'cv_error']]
y = merged['pcreg_wins'].astype(int)

tree = DecisionTreeClassifier(max_depth=4, min_samples_leaf=50)
tree.fit(X_observable, y)

print("Decision Tree with Observable Factors Only:")
print("="*60)
print(export_text(tree, feature_names=['n_lots', 'correlation', 'cv_error']))

print("\nFeature Importances:")
for name, imp in zip(['n_lots', 'correlation', 'cv_error'], tree.feature_importances_):
    print(f"  {name}: {imp:.3f}")

print("\nKey Finding: cv_error is the strongest predictor (65% importance)!")

## 7. Model Performance Summary

In [ ]:
# Overall performance by model
performance = df.groupby('model_name').agg({
    'test_sspe': ['mean', 'std', 'median'],
    'b_error': 'mean',
    'c_error': 'mean',
    'b_correct_sign': 'mean',
    'c_correct_sign': 'mean'
}).round(4)

performance.columns = ['Mean SSPE', 'Std SSPE', 'Median SSPE', 'Mean b_error', 'Mean c_error', 'b Correct Sign %', 'c Correct Sign %']
performance = performance.sort_values('Mean SSPE')

print("Model Performance Summary (sorted by Mean SSPE):")
display(performance)

In [ ]:
# Winner counts by metric
def count_wins(metric, ascending=True):
    """Count how many times each model wins for a given metric."""
    wins = {}
    for key, group in df.groupby(['n_lots', 'target_correlation', 'cv_error', 'learning_rate', 'rate_effect', 'replication']):
        if ascending:
            winner = group.loc[group[metric].idxmin(), 'model_name']
        else:
            winner = group.loc[group[metric].idxmax(), 'model_name']
        wins[winner] = wins.get(winner, 0) + 1
    return pd.Series(wins).sort_values(ascending=False)

print("Winner Counts for test_sspe (lower is better):")
sspe_wins = count_wins('test_sspe', ascending=True)
for model, count in sspe_wins.head(5).items():
    print(f"  {model}: {count:,} wins")

## 8. Practical Recommendations

### When to Use PCReg

1. **Always consider PCReg** when you have domain knowledge to specify reasonable coefficient bounds

2. **Strongly prefer PCReg** when:
   - Data quality is high (low residual variance): win rate 67-75%
   - Sample size is small (n < 15 lots): win rate 57-75%
   - You suspect OLS may produce wrong signs: win rate 81%

3. **Consider OLS** when:
   - Data quality is poor (high noise) AND sample size is large (n >= 30)
   - PCReg win rate drops to 34% in this scenario

### Implementation Guidance

1. **Start with constraints only** (alpha=0) - this outperforms CV-tuned penalties 62% of the time

2. **Use loose bounds** (e.g., -0.5 to 0 for learning curve slopes) rather than trying to specify tight bounds

3. **If using penalty**: Use very small alpha values (0.0001 to 0.001)

In [ ]:
# Summary decision table
summary = pd.DataFrame({
    'Condition': [
        'cv_error = 0.01 (high quality)',
        'cv_error = 0.10, n <= 10',
        'cv_error = 0.10, n = 30',
        'cv_error = 0.20, n = 5',
        'cv_error = 0.20, n = 10',
        'cv_error = 0.20, n = 30',
        'OLS has wrong sign'
    ],
    'PCReg Win Rate': ['67-75%', '57-64%', '48%', '58%', '47%', '34%', '81%'],
    'Recommendation': [
        'Use PCReg',
        'Use PCReg',
        'No clear winner',
        'Slight edge to PCReg',
        'No clear winner',
        'Consider OLS',
        'Use PCReg'
    ]
})

print("Decision Table:")
display(summary)

## Appendix: Simulation Design

- **Factors**: 5 (n_lots, correlation, cv_error, learning_rate, rate_effect)
- **Levels per factor**: 3
- **Total scenarios**: 243
- **Replications per scenario**: 25
- **Total model fits**: 60,750
- **Test data**: 5 out-of-sample lots per replication

### Data Generating Process

$$Y = T_1 \cdot X_1^b \cdot X_2^c \cdot \exp(\epsilon)$$

Where:
- $T_1 = 100$ (first unit cost)
- $b$ = learning curve slope (varies by learning_rate)
- $c$ = rate effect slope (varies by rate_effect)
- $X_1$ = lot midpoint
- $X_2$ = lot quantity (rate variable)
- $\epsilon \sim N(0, \sigma^2)$ where $\sigma^2 = \log(1 + cv\_error^2)$